In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests
import time
from yahooquery import Ticker

In [ ]:
import pandas as pd
import requests
from yahooquery import Ticker

# ==========================================
# 1. FETCH UNIVERSE (Raw List) for Canadian Markets
# ==========================================
def get_canadian_universe_robust():
    print("--- STEP 1: Fetching TSX & TSX-V Stock List ---")
    tickers = []
    
    # 1. Try Fetching from TMX
    url = "https://www.tsx.com/files/trading/moc-eligible-stocks.txt"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            lines = response.text.split('\n')
            for line in lines:
                parts = line.split()
                if len(parts) < 2: continue
                
                exchange = parts[0]
                symbol = parts[1]
                
                # Yahoo Format Conversion
                if exchange == 'TSX':
                    tickers.append(f"{symbol.replace('.', '-')}.TO")
                elif exchange == 'TSXV':
                    tickers.append(f"{symbol.replace('.', '-')}.V")
                    
        print(f"   -> Scraper found {len(tickers)} candidates.")
    except Exception as e:
        print(f"   -> Web scrape failed ({e}).")

    # 2. Backup List
    if len(tickers) == 0:
        print("   -> Using Backup List...")
        tickers = [
            'SHOP.TO', 'CSU.TO', 'ATD.TO', 'DOL.TO', 'L.TO', 'WN.TO', 'EMP-A.TO', 'MRU.TO',
            'CNR.TO', 'CP.TO', 'TFII.TO', 'WCN.TO', 'CAE.TO', 'AC.TO',
            'SU.TO', 'CNQ.TO', 'CVE.TO', 'IMO.TO', 'TOU.TO', 'ARX.TO',
            'NTR.TO', 'TECK-B.TO', 'FM.TO', 'CCO.TO', 'WPM.TO', 'AEM.TO', 'ABX.TO',
            'OTEX.TO', 'GIB-A.TO', 'KXS.TO', 'DSG.TO',
            'BCE.TO', 'T.TO', 'RCI-B.TO', 'QBR-B.TO',
            'FTS.TO', 'EMA.TO', 'H.TO', 'AQN.TO', 'NPI.TO'
        ]

    return list(set(tickers))

# ==========================================
# 2. BUFFETT SCAN (With Price/Vol Filters)
# ==========================================
def run_buffett_nav_scan_filtered(ticker_list):
    """
    Scans for companies trading BELOW Book Value (P/B < 1) 
    that are still profitable (ROE > 0) AND meet liquidity requirements.
    """
    print(f"\n--- STEP 2: Running 'Buffett NAV' Filter ---")
    print("   Criteria: Price > $3 | Cap > $50M | Vol > 50k")
    print("   Value Criteria: P/B < 1.0 | ROE > 0% | Debt/Eq < 100%")
    
    buffett_picks = []
    chunk_size = 300 
    
    for i in range(0, len(ticker_list), chunk_size):
        chunk = ticker_list[i:i+chunk_size]
        print(f"   Scanning batch {i} - {min(i+chunk_size, len(ticker_list))}...", end='\r')
        
        try:
            # Asynchronous fetch
            yq = Ticker(chunk, asynchronous=True)
            data = yq.get_modules("defaultKeyStatistics financialData price summaryProfile summaryDetail")
            
            for symbol in chunk:
                if symbol not in data or isinstance(data[symbol], str): continue
                
                # --- DATA EXTRACTION ---
                price_mod = data[symbol].get('price', {})
                summ_det = data[symbol].get('summaryDetail', {})
                stats = data[symbol].get('defaultKeyStatistics', {})
                fin = data[symbol].get('financialData', {})
                profile = data[symbol].get('summaryProfile', {})
                
                # --- 1. PRE-FILTERS (Liquidity & Size) ---
                price = price_mod.get('regularMarketPrice', 0) or 0
                mkt_cap = price_mod.get('marketCap', 0) or 0
                avg_vol = summ_det.get('averageVolume', 0) or 0
                
                # FILTER: Price > $3
                if price < 3.0: continue
                
                # FILTER: Market Cap > $50 Million
                if mkt_cap < 50_000_000: continue
                
                # FILTER: Avg Volume > 50,000
                if avg_vol < 50_000: continue

                # --- 2. VALUE CRITERIA (Buffett Logic) ---
                
                # P/B < 1.0 (Trading under NAV)
                pb_ratio = stats.get('priceToBook')
                if pb_ratio is None: continue
                if pb_ratio >= 1.0: continue 
                if pb_ratio <= 0: continue # Skip insolvent
                
                # ROE > 0 (Must be Profitable)
                roe = fin.get('returnOnEquity', 0) or 0
                if roe <= 0: continue
                
                # Debt/Equity < 100% (Safety)
                debt_equity = fin.get('debtToEquity', 0) or 0
                if debt_equity > 100: continue 

                # --- CAPTURE ---
                sector = profile.get('sector', 'Unknown')
                
                buffett_picks.append({
                    'Ticker': symbol,
                    'Price': price,
                    'P/B Ratio': round(pb_ratio, 2),
                    'ROE %': round(roe * 100, 2),
                    'Debt/Eq %': round(debt_equity, 2),
                    'Market Cap (M)': round(mkt_cap / 1_000_000, 1),
                    'Vol': avg_vol,
                    'Sector': sector
                })
                        
        except Exception as e:
            continue
            
    df = pd.DataFrame(buffett_picks)
    
    if not df.empty:
        print(f"\n   -> Found {len(df)} Buffett-style value plays.")
        # Sort by P/B Ratio (Deepest Value First)
        return df.sort_values(by='P/B Ratio', ascending=True)
    else:
        print(f"\n   -> No stocks found matching criteria.")
        return pd.DataFrame()

# ==========================================
# MAIN EXECUTION
# ==========================================

# 1. Get Raw Universe
raw_tickers = get_canadian_universe_robust()

# 2. Run Filtered Buffett Scan
buffett_results = run_buffett_nav_scan_filtered(raw_tickers)

# 3. Display Results
if not buffett_results.empty:
    print("\n\n" + "="*60)
    print("WARREN BUFFETT SCREEN (Price > $3, Cap > $50M, Vol > 50k)")
    print("="*60)
    
    # Display columns
    cols = ['Ticker', 'Price', 'P/B Ratio', 'ROE %', 'Debt/Eq %', 'Market Cap (M)', 'Vol', 'Sector']
    
    # Show top 25 results
    try:
        display(buffett_results[cols].head(25))
    except:
        print(buffett_results[cols].head(25))
else:
    print("No results found.")

--- STEP 1: Fetching TSX & TSX-V Stock List ---
   -> Scraper found 0 candidates.
   -> Using Backup List...

--- STEP 2: Running 'Buffett NAV' Filter ---
   Criteria: Price > $3 | Cap > $50M | Vol > 50k
   Value Criteria: P/B < 1.0 | ROE > 0% | Debt/Eq < 100%
   Scanning batch 0 - 40...
   -> No stocks found matching criteria.
No results found.
